# Small branch-and-bound example

Solve the following MIP with branch-and-bound:

$
\begin{align}
\max_x & 15x_1 + 12x_2 + 4x_3 + 2x_4\\
\text{s.t.} & 8x_1 + 5x_2 + 3x_3+ 2x_4 \leq 10\\
& x_i \in \mathbb{Z}_+  \ \forall i=1,2,3,4
\end{align}$

Note that while it is possible to do this by hand, Gurobi is doing exactly this (just a little more cleverly) whenever we give it a general MIP to solve!

In [1]:
using JuMP, Clp

m = Model(Clp.Optimizer)
set_optimizer_attribute(m, "LogLevel",0)
@variable(m, x[1:4] >= 0)

@objective(m, Max, 15x[1] + 12x[2] + 4x[3] + 2x[4])

@constraint(m, x .<= ones(4)) # handy way to give a vector of upper bounds as a constraint
@constraint(m, 8x[1] + 5x[2] + 3x[3] + 2x[4] <= 10)

optimize!(m)

println("LP relaxation (node 1)")
println(objective_value(m))
println(value.(x))

# x1 is fractional: branch and solve problem with x_1 <= 0

@constraint(m, x[1] <= 0)

optimize!(m)

println("x1 <= 0 (node 2)")

println(objective_value(m))
println(value.(x))

# feasible, new lower bound z = 18. solve problem with x_1 >= 1

# reset model to remove constraints added in previous branch
m = Model(Clp.Optimizer)
set_optimizer_attribute(m, "LogLevel",0)

@variable(m, x[1:4] >= 0)
@objective(m, Max, 15x[1] + 12x[2] + 4x[3] + 2x[4])
@constraint(m, x .<= ones(4))
@constraint(m, x[1] >= 1)
@constraint(m, 8x[1] + 5x[2] + 3x[3] + 2x[4] <= 10)

optimize!(m)

println("x1 >= 1 (node 3)")
println(objective_value(m))
println(value.(x))

# x2 fractional: branch and solve x2 <= 0

@constraint(m, x[2] <= 0)

optimize!(m)

println("x1 >= 1, x2 <= 0 (node 4)")
println(objective_value(m))
println(value.(x))

# worse than best lower bound. adding constraints will only make it worse. prune.
# solve problem with x2 >= 1

# reset model to remove constraints added in previous branch
m = Model(Clp.Optimizer)
set_optimizer_attribute(m, "LogLevel",0)

@variable(m, x[1:4] >= 0)
@objective(m, Max, 15x[1] + 12x[2] + 4x[3] + 2x[4])
@constraint(m, x .<= ones(4))
@constraint(m, x[1] >= 1)
@constraint(m, x[2] >= 1)
@constraint(m, 8x[1] + 5x[2] + 3x[3] + 2x[4] <= 10)

optimize!(m)

println("x1 >= 1, x2>= 1 (node 5)")
println(objective_value(m))
println(value.(x))

# infeasible! prune node 5. node 2 gave us the optimal solution:
# z = 18, x = [0, 1, 1, 1]

LP relaxation (node 1)
21.375
[0.625, 1.0, 0.0, 0.0]
x1 <= 0 (node 2)
18.0
[0.0, 1.0, 1.0, 1.0]
x1 >= 1 (node 3)
19.8
[1.0, 0.4, 0.0, 0.0]
x1 >= 1, x2 <= 0 (node 4)
17.666666666666668
[1.0, 0.0, 0.6666666666666666, 0.0]
x1 >= 1, x2>= 1 (node 5)
19.799999999999997


LoadError: Primal solution not available